In [2]:
import numpy as np 
import pandas as pd 
from collections import Counter 
from faker import Faker 
from ortools.sat.python import cp_model 
from ortools.linear_solver import pywraplp 



In [3]:
# Define dict of options
# Format: {option: max_capacity}
options_dict = {
    "Badminton": 20,
    "Football 1": 28,
    "Football 2": 28,
    "Basketball": 15,
    "Cricket": 22,
    "Basket Weaving": 20,
    "Lego Engineering": 20,
    "History Guided Reading": 20,
    "Learn a Brass Instrument": 10,
    "Public Speaking": 15,
    "Knitting": 15,
    "Karate": 15,
}

# Convert to DataFrame and insert id column
options = pd.DataFrame(list(options_dict.items()), columns=['Option', 'Maximum capacity'])
options.insert(0, 'option_id', range(1, len(options)+1))
options

,option_id,Option,Maximum capacity
0,1,Badminton,20
1,2,Football 1,28
2,3,Football 2,28
3,4,Basketball,15
4,5,Cricket,22
5,6,Basket Weaving,20
6,7,Lego Engineering,20
7,8,History Guided Reading,20
8,9,Learn a Brass Instrument,10
9,10,Public Speaking,15


In [4]:
student_ids = [_ for _ in range(1, 201)] 

fake = Faker() 
student_names = [f"{fake.first_name()} {fake.last_name()}" for _ in student_ids] 
students = pd.DataFrame({
    'student_id':student_ids,
    'student_name':student_names,
}) 
students

,student_id,student_name
0,1,Holly Strong
1,2,Lucas Thomas
2,3,Brianna Robinson
3,4,Sarah Cordova
4,5,Paul Rios
...,...,...
195,196,Elizabeth Velasquez
196,197,Samuel Cruz
197,198,Carolyn Miller
198,199,William Chaney


In [6]:
preferences_dict = {student: np.random.choice(options['option_id'], size=4, replace=False) for student in student_ids}
preferences = pd.DataFrame.from_dict(preferences_dict, orient='index', columns=[f'Preference {i}' for i in range(1, 4+1)])
preferences.index.name = 'student_id'
preferences.reset_index(inplace=True)

# Merge
students = pd.merge(students, preferences, on='student_id')
students

,student_id,student_name,Preference 1,Preference 2,Preference 3,Preference 4
0,1,Holly Strong,4,7,6,11
1,2,Lucas Thomas,11,1,4,3
2,3,Brianna Robinson,1,11,12,5
3,4,Sarah Cordova,4,10,8,6
4,5,Paul Rios,10,7,6,3
...,...,...,...,...,...,...
195,196,Elizabeth Velasquez,11,7,1,2
196,197,Samuel Cruz,10,1,11,7
197,198,Carolyn Miller,4,2,8,6
198,199,William Chaney,12,11,9,7


In [7]:
num_students = students.shape[0] 
num_electives = options.shape[0] 

print(f"Number of students:{num_students}") 
print(f"Number of electives:{num_electives}") 

assert sum(options_dict.values()) >= len(students) 


Number of students:200
Number of electives:12


In [8]:
solver = pywraplp.Solver.CreateSolver('SCIP')

In [9]:
decis_vars = {} 
for i in students['student_id']:
    for j in options['option_id']:
        decis_vars[f'student{i}_elective{j}'] = solver.BoolVar(f'student{i}_elective{j}') 
print(f"Number of decision variables = {solver.NumVariables()}")

Number of decision variables = 2400


In [10]:
# Define the constraints

# 1. Constraint #1: Each student can only be assigned to one elective option
for student_id in students['student_id']:
    
    # Collect the decision variables related to that student
    relevant_variables = []
    for d in decis_vars.keys():
        if d.startswith(f'student{student_id}_'):
            relevant_variables.append(decis_vars[d])

    # Add the constraint
    solver.Add(sum(relevant_variables) == 1)
    
    
# 2. Constraint #2: Each student must be assigned to one of their top 4 preferences
for student_id in students['student_id']:
    
    # Identify the student's top preferences
    preferences = list(preferences_dict[student_id])
    
    relevant_variables = []
    for p in preferences:
        for d in decis_vars.keys():
            if d.startswith(f'student{student_id}_') and d.endswith(f'elective{p}'):
                relevant_variables.append(decis_vars[d])

    # Add the constraint
    solver.Add(sum(relevant_variables) == 1)
    
for option in options['option_id']:
    max_num_students = options[options['option_id']==option]['Maximum capacity'].values[0]
    relevant_variables = []
    for d in decis_vars.keys():
        if d.endswith(f'elective{option}'):
            relevant_variables.append(decis_vars[d])
            
    # Add the constraint
    solver.Add(sum(relevant_variables) <= max_num_students)

In [13]:
objective = solver.Objective() 
for index,row in students.iterrows():
    student_id = row['student_id'] 
    for pref_rank in range(1,5):
        preference = row[f'Preference {pref_rank}'] 
        for option_index,option_row in options.iterrows():
            option_id = option_row['option_id'] 
            if option_id == preference:
                var_name = f'student{student_id}_elective{option_id}' 
                if var_name in decis_vars:
                    var = decis_vars[var_name] 
                    objective.SetCoefficient(var,5 - pref_rank) 
objective.SetMaximization()

In [14]:
status = solver.Solve() 
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal solution found!') 
    print('Objective value =',solver.Objective().Value()) 


Optimal solution found!
Objective value = 782.0


In [15]:
for var in decis_vars.values():
    print(var.solution_value())

0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [16]:
def get_relevant_vars(student_id):
    """Find the (12) decision variables related to a given `student_id`."""
    return {k: v for k, v in decis_vars.items() if f'student{student_id}' in k}

def show_elective(list_of_vars):
    """For a given list of decision variables, find the one(s) equal to 1."""
    vars_equal_to_1 = [k.split('elective')[1] for k, v in list_of_vars.items() if v.solution_value() == 1]
    return vars_equal_to_1[0]

def get_final_allocations(list_of_student_ids=students['student_id']):
    """For a given `list_of_student_ids`, retrieve the final allocations."""
    final_allocations = {student: show_elective(get_relevant_vars(student)) for student in list_of_student_ids}
    return final_allocations
        
def count_final_allocations(allocations_dict):
    """Count the number of student allocated to each elective."""
    return Counter(allocations_dict.values())

def count_num_students_who_got_each_choice():
    """Count how many students got their 1st choice, how many got their second, etc."""       
    
    satisfaction = {}
    for student in students['student_id']:
        allocated_elective = final_allocations[student]
        
        for i in range(1, 5):
            choice = students[students['student_id']==student][f'Preference {i}'].values[0]
            
            if allocated_elective == str(choice):
                satisfaction[student] = i 
    return satisfaction

In [17]:
final_allocations = get_final_allocations() 
final_allocations

{1: '7',
 2: '11',
 3: '1',
 4: '4',
 5: '7',
 6: '2',
 7: '11',
 8: '1',
 9: '2',
 10: '3',
 11: '11',
 12: '12',
 13: '12',
 14: '10',
 15: '9',
 16: '1',
 17: '5',
 18: '3',
 19: '9',
 20: '8',
 21: '2',
 22: '12',
 23: '6',
 24: '2',
 25: '12',
 26: '6',
 27: '3',
 28: '5',
 29: '10',
 30: '9',
 31: '10',
 32: '9',
 33: '2',
 34: '5',
 35: '4',
 36: '2',
 37: '11',
 38: '5',
 39: '12',
 40: '12',
 41: '7',
 42: '8',
 43: '9',
 44: '5',
 45: '11',
 46: '5',
 47: '2',
 48: '8',
 49: '4',
 50: '9',
 51: '5',
 52: '10',
 53: '3',
 54: '2',
 55: '5',
 56: '2',
 57: '3',
 58: '1',
 59: '5',
 60: '8',
 61: '6',
 62: '2',
 63: '7',
 64: '8',
 65: '9',
 66: '7',
 67: '6',
 68: '6',
 69: '2',
 70: '5',
 71: '1',
 72: '6',
 73: '8',
 74: '12',
 75: '1',
 76: '7',
 77: '4',
 78: '10',
 79: '10',
 80: '3',
 81: '3',
 82: '6',
 83: '6',
 84: '4',
 85: '8',
 86: '10',
 87: '11',
 88: '11',
 89: '3',
 90: '3',
 91: '4',
 92: '5',
 93: '1',
 94: '4',
 95: '6',
 96: '1',
 97: '1',
 98: '11',
 99: '1

In [18]:
count_final_allocations(final_allocations)

Counter({'2': 21,
         '1': 20,
         '5': 20,
         '8': 20,
         '6': 18,
         '3': 17,
         '7': 16,
         '4': 15,
         '12': 15,
         '10': 15,
         '11': 13,
         '9': 10})

In [20]:
performance = Counter(count_num_students_who_got_each_choice().values()) 
for i in range(1,5):
    print(f"the number of students who got their #{i} choice was {performance[i]}")

the number of students who got their #1 choice was 182
the number of students who got their #2 choice was 18
the number of students who got their #3 choice was 0
the number of students who got their #4 choice was 0


In [21]:
preferences_dict = {student: np.random.choice(options['option_id'], size=4, replace=False) for student in student_ids}

In [22]:
preferences_dict = {student: np.random.choice(options['option_id'], size=4, replace=False, p=[0.05, 0.25, 0.25, 0.1, 0.05, 0.05, 0.05, 0.05, 0.1, 0.01, 0.01, 0.03]) for student in student_ids}

In [23]:
temp_df = students['Preference 1'].value_counts().to_frame() 
temp_df.index.name = 'option_id' 
temp_df.reset_index(inplace = True) 
pd.merge(
    temp_df,
    options,
    on='option_id'
)

,option_id,count,Option,Maximum capacity
0,8,22,History Guided Reading,20
1,4,20,Basketball,15
2,1,20,Badminton,20
3,10,20,Public Speaking,15
4,5,20,Cricket,22
5,12,19,Karate,15
6,2,16,Football 1,28
7,3,15,Football 2,28
8,11,12,Knitting,15
9,9,12,Learn a Brass Instrument,10


In [24]:
performance = Counter(count_num_students_who_got_each_choice().values())

for i in range(1, 5):
    print(f"The number of students who got their #{i} choice was {performance[i]}")


The number of students who got their #1 choice was 182
The number of students who got their #2 choice was 18
The number of students who got their #3 choice was 0
The number of students who got their #4 choice was 0


In [25]:
min_num_students = 5
for option in options['option_id']:
    
    relevant_variables = []
    for d in decis_vars.keys():
        if d.endswith(f'elective{option}'):
            relevant_variables.append(decis_vars[d])
            
    # Add the constraint
    solver.Add(sum(relevant_variables) >= min_num_students)